In [1]:
import networkx as nx
from gnpsdata import taskresult
import os
from gnpsdata import workflow_fbmn
import pandas as pd

In [2]:
task = "cf6e14abf5604f47b28b467a513d3532"

In [3]:
# Making sure the data folder exists
os.makedirs("../data", exist_ok=True)

# Downloading GNPS Qiime2 object
workflow_fbmn.download_qiime2(task, "../data/qiime_table.qza")

# Downloading metadata
workflow_fbmn.download_metadata(task, "../data/unprocessed_metadata.tsv")

# Changing Metadata Column Name

In [4]:
#read metadata file
metadata = pd.read_csv("../data/unprocessed_metadata.tsv", sep = "\t", index_col=False)
#rename 1st column to "#OTU ID
metadata = metadata.rename(columns={"filename":"#OTU ID"})
#convert back to .tsv
metadata.to_csv('../data/metadata.tsv', sep="\t", index=False)

# ANOVA
! source activate qiime2-2019.4 && qiime longitudinal anova \
  --m-metadata-file <path-to-metadata-file.tsv> \
  --p-formula "independent_variable~dependent_variable+dependent_variable" \
  --o-visualization <path-to-visualization.qza>

In [5]:
!source activate qiime2-2019.4 && qiime longitudinal anova \
  --m-metadata-file ../data/metadata.tsv \
  --p-formula "ATTRIBUTE_Year~ATTRIBUTE_Sample_Area+ATTRIBUTE_Latitude" \
  --p-sstype 'I' \
  --o-visualization ../data/metadata.qzv

/bin/bash: line 1: activate: No such file or directory


# Visualization
Qiime2 visualizations do not work in headless environments, we can view them at https://view.qiime2.org/

In [13]:
# Qiime2 Manipulations Happen after this to mirror the other notebooks

! conda activate qiime2-2019.4 && qiime

no change     /home/user/miniconda3/condabin/conda
no change     /home/user/miniconda3/bin/conda
no change     /home/user/miniconda3/bin/conda-env
no change     /home/user/miniconda3/bin/activate
no change     /home/user/miniconda3/bin/deactivate
no change     /home/user/miniconda3/etc/profile.d/conda.sh
no change     /home/user/miniconda3/etc/fish/conf.d/conda.fish
no change     /home/user/miniconda3/shell/condabin/Conda.psm1
no change     /home/user/miniconda3/shell/condabin/conda-hook.ps1
no change     /home/user/miniconda3/lib/python3.9/site-packages/xontrib/conda.xsh
no change     /home/user/miniconda3/etc/profile.d/conda.csh
no change     /home/user/.bashrc
No action taken.

CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPOR

# Principal Coordinate Analysis (PCoA) & Distance Matrix
! source activate qiime2-2019.4 && first create distance matrix 
 qiime diversity beta \
  --i-table <path-to-feature-table.qza> \
  --p-metric canberra_adkins \
  --output-dir <path-to-distance-matrix.qza> \

In [7]:
! source activate qiime2-2019.4 && qiime diversity beta \
  --i-table ../data/qiime_table.qza \
  --p-metric canberra_adkins \
  --o-distance-matrix ../data/distance_matrix.qza

/bin/bash: line 1: activate: No such file or directory


## PCoA
! source activate qiime2-2019.4 && qiime diversity pcoa \
  --i-distance-matrix <path-to-distance-matrix.qza> \
  --p-number-of-dimensions INTEGER \ #optional
  --o-pcoa <path-to-artifact.qza>

In [20]:
! source activate qiime2-2019.4 && qiime diversity pcoa \
  --i-distance-matrix ../data/distance_matrix.qza \
  --o-pcoa ../data/pcoa.qza

/opt/conda/envs/qiime2-2019.4/lib/python3.6/site-packages/skbio/util/_testing.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as pdt
Saved PCoAResults to: ../data/pcoa.qza


# Emperor plot
! source activate qiime2-2019.4 && qiime emperor plot \
--i-pcoa <path_to_pcoa.qza> \
--m-metadata-file <path_to_metadata.tsv \
--o-visualization emperor_qiime2

In [21]:
! source activate qiime2-2019.4 && qiime emperor plot \
  --i-pcoa ../data/pcoa.qza \
  --m-metadata-file ../data/metadata.tsv \
  --o-visualization ../data/emperor_plot.qzv

/opt/conda/envs/qiime2-2019.4/lib/python3.6/site-packages/skbio/util/_testing.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as pdt
Saved Visualization to: ../data/emperor_plot.qzv


In [ ]:
# Visualization

from qiime2 import Visualization
Visualization.load('../data/emperor_plot.qzv')